In [219]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `~/Desktop/OptimalC/lecture-notebooks/Lecture 12/Project.toml`


In [220]:
using LinearAlgebra
Pkg.add("PyPlot")
using PyPlot
Pkg.add("ForwardDiff")
using ForwardDiff
Pkg.add("RobotZoo")
using RobotZoo
Pkg.add("RobotDynamics")
using RobotDynamics
Pkg.add("Ipopt")
using Ipopt
using MathOptInterface
const MOI = MathOptInterface;

   Resolving package versions...


  No Changes to `~/Desktop/OptimalC/lecture-notebooks/Lecture 12/Project.toml`
  No Changes to `~/Desktop/OptimalC/lecture-notebooks/Lecture 12/Manifest.toml`


   Resolving package versions...


  No Changes to `~/Desktop/OptimalC/lecture-notebooks/Lecture 12/Project.toml`
  No Changes to `~/Desktop/OptimalC/lecture-notebooks/Lecture 12/Manifest.toml`


   Resolving package versions...


  No Changes to `~/Desktop/OptimalC/lecture-notebooks/Lecture 12/Project.toml`
  No Changes to `~/Desktop/OptimalC/lecture-notebooks/Lecture 12/Manifest.toml`


   Resolving package versions...


  No Changes to `~/Desktop/OptimalC/lecture-notebooks/Lecture 12/Project.toml`
  No Changes to `~/Desktop/OptimalC/lecture-notebooks/Lecture 12/Manifest.toml`


   Resolving package versions...


  No Changes to `~/Desktop/OptimalC/lecture-notebooks/Lecture 12/Project.toml`
  No Changes to `~/Desktop/OptimalC/lecture-notebooks/Lecture 12/Manifest.toml`


In [221]:
function CR3BPdynamics(rv,μ) #Three body dynamics in Earth/Moon System
    x,y,z,vx,vy,vz = rv
    r₁³= ((x + μ)^2     + y^2 + z^2)^1.5; # distance to m1, LARGER MASS
    r₂³= ((x - 1 + μ)^2 + y^2 + z^2)^1.5; # distance to m2, smaller mass

    rvdot = zeros(6)
    rvdot[1:3] = [vx;vy;vz]
    rvdot[4] = -((1 - μ)*(x + μ)/r₁³) - (μ*(x - 1 + μ)/r₂³) + 2*vy + x;
    rvdot[5] = -((1 - μ)*y      /r₁³) - (μ*y          /r₂³) - 2*vx + y;
    rvdot[6] = -((1 - μ)*z      /r₁³) - (μ*z          /r₂³);
    return rvdot
end

CR3BPdynamics (generic function with 2 methods)

In [222]:
#Pendulum Dynamics
a = RobotZoo.Acrobot()
h = 0.1 #10 Hz
μ = 9.537e-4

0.0009537

In [223]:
Nx = 4     # number of state
Nu = 1     # number of controls
Tfinal = 5.0 # final time
Nt = Int(Tfinal/h)+1   # number of time steps
thist = Array(range(0,h*(Nt-1), step=h));
n_nlp = (Nx+Nu)*Nt # number of decision variables
m_nlp = Nx*(Nt+1)

208

In [224]:
# Nx = 6     # number of state
# Nu = 1     # number of controls
# Tfinal = 5.0 # final time
# Nt = Int(Tfinal/h)+1   # number of time steps
# thist = Array(range(0,h*(Nt-1), step=h));
# n_nlp = (Nx+Nu)*Nt # number of decision variables
# m_nlp = Nx*(Nt+1)

In [225]:
function dircol_dynamics(x1,u1,x2,u2)
    #Hermite-Simpson integration with first-order hold on u
    f1 = dynamics(a, x1, u1)
    f2 = dynamics(a, x2, u2)
    xm = 0.5*(x1 + x2) + (h/8.0)*(f1 - f2)


    
    ẋm = (-3/(2.0*h))*(x1 - x2) - 0.25*(f1 + f2)
    fm = dynamics(a, xm, um)
    return fm - ẋm
end

dircol_dynamics (generic function with 2 methods)

In [226]:
function dircol_dynamics(x1,x2,h)
    #Hermite-Simpson integration with first-order hold on u
    f1 = CR3BPdynamics(x1,μ)
    f2 = CR3BPdynamics(x2,μ)
    xm = 0.5*(x1 + x2) + (h/8.0)*(f1 - f2)
    # um = 0.5*(u1 + u2)
    ẋm = (-3/(2.0*h))*(x1 - x2) - 0.25*(f1 + f2)
    fm = CR3BPdynamics(xm,μ)
    return fm - ẋm
end

In [227]:
# Cost weights
Q = Diagonal([1.0*ones(2); 1.0*ones(2)]);
R = 0.1;

In [228]:
# # Cost weights
# Q = Diagonal([1.0*ones(3); 1.0*ones(3)]);
# R = 0.1;

In [229]:
function stage_cost(x,u)
    return 0.5*((x-xgoal)'*Q*(x-xgoal)) + 0.5*u'*R*u
end

stage_cost (generic function with 1 method)

In [230]:
function cost(ztraj)
    z = reshape(ztraj,Nx+Nu,Nt)
    J = 0.0
    for k = 1:Nt
        J += stage_cost(z[1:Nx,k],z[(Nx+1):(Nx+Nu),k])
    end
    return J
end

cost (generic function with 1 method)

In [231]:
function dynamics_constraint!(c,ztraj)
    d = reshape(c,Nx,Nt-1)
    z = reshape(ztraj,Nx+Nu,Nt)
    for k = 1:(Nt-1)
        x1 = z[1:Nx,k]
        u1 = z[(Nx+1):(Nx+Nu),k]
        x2 = z[1:Nx,k+1]
        u2 = z[(Nx+1):(Nx+Nu),k+1]
        d[:,k] = dircol_dynamics(x1,u1,x2,u2)
    end
    return nothing
end

dynamics_constraint! (generic function with 1 method)

In [232]:
function con!(c,ztraj)
    z = reshape(ztraj,Nx+Nu,Nt)
    c[1:Nx] .= z[1:Nx,1] - x0
    @views dynamics_constraint!(c[(Nx+1):(end-Nx)],ztraj)
    c[(end-Nx+1):end] .= z[1:Nx,end] - xgoal
end

con! (generic function with 1 method)

In [233]:
function con!(c,ztraj)
    z = reshape(ztraj,Nx+Nu,Nt)
    #Periodicity Constraint 
    #Xf - X0 = 0 
    c[1:Nx] .= z[1:Nx,end] - z[1:Nx,1]
    @views dynamics_constraint!(c[(Nx+1):(end-Nx)],ztraj)
end

con! (generic function with 1 method)

In [234]:
struct ProblemMOI <: MOI.AbstractNLPEvaluator
    n_nlp::Int
    m_nlp::Int
    idx_ineq
    obj_grad::Bool
    con_jac::Bool
    sparsity_jac
    sparsity_hess
    primal_bounds
    constraint_bounds
    hessian_lagrangian::Bool
end

function ProblemMOI(n_nlp,m_nlp;
        idx_ineq=(1:0),
        obj_grad=true,
        con_jac=true,
        sparsity_jac=sparsity_jacobian(n_nlp,m_nlp),
        sparsity_hess=sparsity_hessian(n_nlp,m_nlp),
        primal_bounds=primal_bounds(n_nlp),
        constraint_bounds=constraint_bounds(m_nlp,idx_ineq=idx_ineq),
        hessian_lagrangian=false)

    ProblemMOI(n_nlp,m_nlp,
        idx_ineq,
        obj_grad,
        con_jac,
        sparsity_jac,
        sparsity_hess,
        primal_bounds,
        constraint_bounds,
        hessian_lagrangian)
end

function primal_bounds(n)
    x_l = -Inf*ones(n)
    x_u = Inf*ones(n)
    return x_l, x_u
end

#Our code 
#Change bounds to zero.
#Why infinity? 
function primal_bounds(n)
    x_l = zeros(n)
    x_u = zeros(n)
    return x_l, x_u
end
#Our code 
function constraint_bounds(m; idx_ineq=(1:0))
    c_l = zeros(m)
    c_l[idx_ineq] .= -Inf

    c_u = zeros(m)
    return c_l, c_u
end

#Our code 
function constraint_bounds(m; idx_ineq=(1:0))
    c_l = zeros(m)
    #Check idx_ineq??? 
    c_l[idx_ineq] .= 0
#Variable upper bound 
    c_u = 100*ones(m)
    return c_l, c_u
end

#Our code 

function row_col!(row,col,r,c)
    for cc in c
        for rr in r
            push!(row,convert(Int,rr))
            push!(col,convert(Int,cc))
        end
    end
    return row, col
end

function sparsity_jacobian(n,m)

    row = []
    col = []

    r = 1:m
    c = 1:n

    row_col!(row,col,r,c)

    return collect(zip(row,col))
end

function sparsity_hessian(n,m)

    row = []
    col = []

    r = 1:m
    c = 1:n

    row_col!(row,col,r,c)

    return collect(zip(row,col))
end

function MOI.eval_objective(prob::MOI.AbstractNLPEvaluator, x)
    cost(x)
end

function MOI.eval_objective_gradient(prob::MOI.AbstractNLPEvaluator, grad_f, x)
    ForwardDiff.gradient!(grad_f,cost,x)
    return nothing
end

function MOI.eval_constraint(prob::MOI.AbstractNLPEvaluator,g,x)
    con!(g,x)
    return nothing
end

function MOI.eval_constraint_jacobian(prob::MOI.AbstractNLPEvaluator, jac, x)
    ForwardDiff.jacobian!(reshape(jac,prob.m_nlp,prob.n_nlp), con!, zeros(prob.m_nlp), x)
    return nothing
end

function MOI.features_available(prob::MOI.AbstractNLPEvaluator)
    return [:Grad, :Jac]
end

MOI.initialize(prob::MOI.AbstractNLPEvaluator, features) = nothing
MOI.jacobian_structure(prob::MOI.AbstractNLPEvaluator) = prob.sparsity_jac

function solve(x0,prob::MOI.AbstractNLPEvaluator;
        tol=1.0e-6,c_tol=1.0e-6,max_iter=10000)
    x_l, x_u = prob.primal_bounds
    c_l, c_u = prob.constraint_bounds

    nlp_bounds = MOI.NLPBoundsPair.(c_l,c_u)
    block_data = MOI.NLPBlockData(nlp_bounds,prob,true)

    solver = Ipopt.Optimizer()
    solver.options["max_iter"] = max_iter
    solver.options["tol"] = tol
    solver.options["constr_viol_tol"] = c_tol

    x = MOI.add_variables(solver,prob.n_nlp)

    for i = 1:prob.n_nlp
        xi = MOI.SingleVariable(x[i])
        MOI.add_constraint(solver, xi, MOI.LessThan(x_u[i]))
        MOI.add_constraint(solver, xi, MOI.GreaterThan(x_l[i]))
        MOI.set(solver, MOI.VariablePrimalStart(), x[i], x0[i])
    end

    # Solve the problem
    MOI.set(solver, MOI.NLPBlock(), block_data)
    MOI.set(solver, MOI.ObjectiveSense(), MOI.MIN_SENSE)
    MOI.optimize!(solver)

    # Get the solution
    res = MOI.get(solver, MOI.VariablePrimal(), x)

    return res
end

solve (generic function with 1 method)

In [235]:
#Initial and goal states
x0 = [-pi/2; 0; 0; 0]
xgoal = [pi/2; 0; 0; 0]

4-element Vector{Float64}:
 1.5707963267948966
 0.0
 0.0
 0.0

In [236]:
#Guess
xguess = kron(ones(Nt)', x0)
# xguess[1,:] .= xguess[1,:] + 0.1*randn(Nt)
uguess = zeros(Nt)'
z0 = reshape([xguess; uguess],(Nx+Nu)*Nt,1);

In [237]:
prob = ProblemMOI(n_nlp,m_nlp)
z_sol = solve(z0,prob) # solve
ztraj = reshape(z_sol,Nx+Nu,Nt)
xtraj = ztraj[1:Nx,:]
utraj = ztraj[(Nx+1):(Nx+Nu),:];

UndefVarError: UndefVarError: SingleVariable not defined

In [238]:
plot(thist,xtraj[1,:])
plot(thist,xtraj[2,:])
plot(thist,xguess[1,:])

UndefVarError: UndefVarError: xtraj not defined

In [239]:
plot(thist,xtraj[3,:])
plot(thist,xtraj[4,:])

UndefVarError: UndefVarError: xtraj not defined

In [240]:
plot(thist,utraj[1,:])

UndefVarError: UndefVarError: utraj not defined

In [241]:
using TrajOptPlots
using MeshCat
using StaticArrays

vis = Visualizer()
render(vis)

ArgumentError: ArgumentError: Package TrajOptPlots not found in current path:
- Run `import Pkg; Pkg.add("TrajOptPlots")` to install the TrajOptPlots package.


In [242]:
TrajOptPlots.set_mesh!(vis, a)

UndefVarError: UndefVarError: TrajOptPlots not defined

In [243]:
X1 = [SVector{4}(x) for x in eachcol(xtraj)];
visualize!(vis, a, thist[end], X1)

UndefVarError: UndefVarError: SVector not defined